<div class="alert alert-block alert-info">
<b>Note:</b> Please run the <b>prepare_data.ipynb</b> notebook in this repo before proceeding onto this one.
</div>

# Fine-tuning and deploying sentence-pair classification model with JumpStart API

We first install the `ipywidgets` for some interactive controls and update our SageMaker SDK version to the latest.

In [2]:
!pip install sagemaker ipywidgets==7 --upgrade --quiet

Keyring is skipped due to an exception: 'keyring.backends'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
sparkmagic 0.20.0 requires nest-asyncio==1.5.5, but you have nest-asyncio 1.5.6 which is incompatible.
docker-compose 1.29.2 requires PyYAML<6,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.24 requires botocore==1.29.24, but you have botocore 1.29.69 which is incompatible.
awscli 1.27.24 requires PyYAML<5.5,>=3.10, but you have pyyaml 6.0 which is incompatible.
awscli 1.27.24 requires rsa<4.8,>=3.1.2, but you have rsa 4.9 which is incompatible.
aiobotocore 2.4.1 requires botocore<1.27.60,>=1.27.59, but you have botocore 1.29.69 which is incompatible.

[notice] A new release of pip av

------
## 1. Set-up permissions and SageMaker role 

If you are going to use Sagemaker in a local environment (not SageMaker Studio or SageMaker Notebook Instances), you will need access to assume an IAM Role with the required permissions for Sagemaker. Find out more about this [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html).

In [4]:
import sagemaker, boto3, json

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = 'sagemaker-studio-075912829265-0uytdt9yg6g'

try:
    aws_role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    aws_role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
aws_region = sess.boto_region_name

print(f"sagemaker role arn: {aws_role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {aws_region}")

sagemaker role arn: arn:aws:iam::075912829265:role/service-role/AmazonSageMaker-ExecutionRole-20230127T081684
sagemaker bucket: sagemaker-studio-075912829265-0uytdt9yg6g
sagemaker session region: us-east-1


----
## 2. Select a pre-trained model

The DistilroBERTa model was pre-selected for the task, but you can choose a different model from the dropdown generated upon running the next cell. The `spc` string within the `model_id` identifies it as a model available for the sentence-pair classification task.

A complete list of JumpStart models can be accessed at [JumpStart Models](https://sagemaker.readthedocs.io/en/stable/doc_utils/jumpstart.html#).



In [5]:
model_id = "huggingface-spc-distilroberta-base"

In [6]:
import IPython
from ipywidgets import Dropdown

# download JumpStart model_manifest file.
boto3.client("s3").download_file(
    f"jumpstart-cache-prod-{aws_region}", "models_manifest.json", "models_manifest.json"
)
with open("models_manifest.json", "rb") as json_file:
    model_list = json.load(json_file)

# filter-out all the Sentence Pair Classification models from the manifest list.
spc_models_all_versions, spc_models = [
    model["model_id"] for model in model_list if "-spc-" in model["model_id"]
], []
[spc_models.append(model) for model in spc_models_all_versions if model not in spc_models]

# display the model-ids in a dropdown, for user to select a model.
dropdown = Dropdown(
    value=model_id,
    options=spc_models,
    description="JumpStart Sentence Pair Classification Models:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
# display(IPython.display.Markdown("### Select a JumpStart pre-trained model from the dropdown below"))
display(dropdown)

A Jupyter Widget

----
## 3. Fine-tune and deploy the model

The models available for fine-tuning do pure Text Embedding, and can be fine-tuned on any sentence pair classification dataset in the same way.
When you fine-tune one of these models, a binary classification layer is attached to the Text Embedding model and the layer parameters are initialized to random values. Then, all the model parameters are fine-tuned to minimize prediction error on the input data, returning a model that can then be deployed for inference. 


### 3.1. Retrieve JumpStart training artifacts  
Here, for the selected model, we retrieve the appropriate training docker container, the training script source, the pre-trained model artifact, and a python dictionary with the training hyper-parameters that the algorithm accepts with their default values. Note that the `model_version="*"` fetches the latest model. Also, we do need to specify the `training_instance_type` to guarantee fetching the right `train_image_uri`.

In [ ]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = dropdown.value, "*"
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

In [ ]:
print(train_image_uri)
print(train_source_uri)
print(train_model_uri)

### 3.2. Set training parameters

There are two kinds of parameters that need to be set for training. 

First are the parameters that define the training job. These include: (i) `training_dataset_s3_path` - this is the S3 folder in which the input data is stored, uploaded when we ran the `prepared_data.ipynb` notebook in this repo , (ii) `s3_output_location` - this is the S3 folder in which the training output is stored, and (iii) `training_instance_type` - this indicates the type of machine on which to run the training; we have defined this last parameter in the ceels above. Typically, we use GPU instances for fine-tuning large models.

The second set of parameters are algorithm specific training hyper-parameters - in this case: epoch, learning rate and batch size.

In [ ]:
training_data_prefix = "datasets/sts-paraphrase/"

training_dataset_s3_path = f"s3://{sagemaker_session_bucket}/{training_data_prefix}"

output_prefix = "training_jobs/jumpstart-example-spc-training"

s3_output_location = f"s3://{sagemaker_session_bucket}/{output_prefix}/output"

In [ ]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparams = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
hyperparams["batch-size"] = "64"
print(hyperparams)

### 3.3. Start Training
Now that all parameters are set, we are ready to fine-tune our Sentence Pair Classification model. To begin, let us create a [``sagemaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object, passing in all the required assets as inputs and then launching the training job with the `.fit` method. 


In [ ]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base

training_job_name = name_from_base(f"jumpstart-example-{model_id}-transfer-learning")

# Create SageMaker Estimator instance
spc_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparams,
    output_path=s3_output_location,
)

# Launch a SageMaker Training job by passing s3 path of the training data
spc_estimator.fit({"training": training_dataset_s3_path}, logs=True)

### 3.4. Deploy the fine-tuned model

A trained model does nothing on its own. We now want to use the model to perform inference.

We start by retrieving the jumpstart artifacts required for deploying an endpoint, and then use them as parameters to deploy our `spc_estimator`.

In [ ]:
inference_instance_type = "ml.m5.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"jumpstart-example-FT-{model_id}")

print(f'Endpoint Name: {endpoint_name}')

In [ ]:
# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = spc_estimator.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

### 3.5 Run inference on deployed endpoint

<div class="alert alert-block alert-info">
<b>Note:</b> To run inference on your model, copy the <b>endpoint_name</b> printed in the cells above and refer to the <b>make_predictions.ipynb</b> notebook in this repo.
</div>


-----
## Clean up

The following cell with delete your deployed endpoint.

In [ ]:
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()